In [0]:
from pyspark.sql import SparkSession, functions as f

df_hosa = spark.read.parquet("/mnt/bronze/hosa/departments")

df_hosb = spark.read.parquet("/mnt/bronze/hosb/departments")

df_merged = df_hosa.unionByName(df_hosb)

df_merged = df_merged.withColumn("SRC_Dept_id", f.col("deptid"))\
    .withColumn("Dept_id",f.concat(f.col("SRC_Dept_id"), f.lit("-"), f.col("deptid")))\
        .drop("deptid")

df_merged.createOrReplaceTempView("deparments")

display(df_merged)

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS `hrcm-data-catalog`.silver;

CREATE TABLE IF NOT EXISTS `hrcm-data-catalog`.silver.departments(
  Dept_id string,
  SRC_Dept_id string,
  Name string,
  Datasource string,
  is_quarantined boolean
)
USING DELTA;

In [0]:
%sql 
TRUNCATE TABLE `hrcm-data-catalog`.silver.departments;

In [0]:
%sql
INSERT INTO `hrcm-data-catalog`.silver.departments
SELECT
Dept_id,
SRC_Dept_id,
Name,
datasource,
CASE WHEN SRC_Dept_id IS NULL OR Name IS NULL  THEN TRUE ELSE FALSE END AS is_quarantined 
FROM deparments;


In [0]:
%sql
select * from `hrcm-data-catalog`.silver.departments;

In [0]:
df_silver = spark.sql("SELECT * FROM `hrcm-data-catalog`.silver.departments")

df_silver.write.format("delta").mode("overwrite").save("/mnt/silver/departments")